In [1]:
using TimeSeries
using Pkg
using DataFrames,DataFramesMeta
using Dates
using Statistics
using Flux
using Flux: @epochs
using Plots
using Hyperopt
using HTTP
using Sockets
using JSON


In [2]:
ENV["PYTHON"] = "/Users/kahingleung/PycharmProjects/mylightning/venv/bin/python3.8"
Pkg.build("PyCall")

   Building Conda ─→ `~/.julia/packages/Conda/x5ml4/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/tqyST/deps/build.log`


In [3]:
using PyCall
function getStock(sym,period)
    yf = pyimport("yfinance")
    #sym = "gbtc"
    #sym = "2840.HK"
    ticker = yf.Ticker(sym)
    etf = ticker.history(period=period)

    function pd_to_df(df_pd)
        df= DataFrame()
        for col in df_pd.columns
            df[!, col] = getproperty(df_pd, col).values
        end
        df[!,:Date] = collect(df_pd[:index])
        return df
    end
    etf = pd_to_df(etf)

    transform!(etf, :Date => ( x -> Date.(Dates.year.(x),Dates.month.(x),Dates.day.(x)) ) => :ondate)
    for c in ["High","Low","Open","Close","Stock Splits"]
        transform!(etf,c => (x -> Float32.(x) )=> c)
    end
    return select(etf,[:Open,:Close,:High,:Low,:Volume,:ondate])
end

getStock (generic function with 1 method)

In [4]:
function wgetStock(sym::String,period::Int)
    fromYear = Dates.format(today()-Year(period), "yyyy-mm-dd")
    headers = ["APCA-API-KEY-ID" => ENV["APCA-API-KEY-ID"], "APCA-API-SECRET-KEY" => ENV["APCA-API-SECRET-KEY"] ]
    url_query = "https://data.alpaca.markets/v1/bars/day?symbols="*sym*"&after="*fromYear*"T00:00:00&limit=1000"
    raw_response = HTTP.request("GET", url_query, headers)
    rsp = JSON.parse(String(raw_response.body))
    df2 = DataFrame(t = Int[],h = Float32[],l = Float32[],o = Float32[],c = Float32[],v = Float32[] )
    for (sym,v) in rsp
        for q in v
            push!(df2,q)
        end
    end
    transform!(df2, :t => ( x -> Dates.unix2datetime.(x) ) => :ondate)
    DataFrames.rename!(df2,:h => :High)
    DataFrames.rename!(df2,:l => :Low)
    DataFrames.rename!(df2,:o => :Open)
    DataFrames.rename!(df2,:c => :Close)
    DataFrames.rename!(df2,:v => :Volume)
    sort!(df2,order(:ondate))
    return select(df2,[:Open,:Close,:High,:Low,:Volume,:ondate])
end

wgetStock (generic function with 1 method)

In [5]:
function toPL(PL_ts)
    function f_PL(values)
        open = values[1]
        close = values[2]
        if close - open > 0
            pl = 1
        else
            pl = 0
        end
        [pl,0]
    end     
    PL = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_PL(values)),PL_ts)[:Open],Symbol("PL") )
    PL = TimeSeries.map((timestamp, values) -> (timestamp, Int(values)),PL)
end

toPL (generic function with 1 method)

In [6]:
function toRSI(price_ts,loopback)
    pct = percentchange(price_ts)
    upidx = findall(pct["Close"] .> 0)
    downidx = findall(pct["Close"] .< 0)
    up = map( (timestamp, values) -> (timestamp, if values < 0 ; 0 ; else ;values ;end), pct)
    down = map( (timestamp, values) -> (timestamp, if values < 0 ; abs(values) ; else ;0 ;end), pct)
    up_roll = moving(mean, up, loopback)
    down_roll = moving(mean, down, loopback)
    function f_rsi(values)
        x = values[1]
        y = values[2]
        rsi = 100 - (100 / (1 + x/y) )
        [rsi,0]
    end
    updown = TimeSeries.rename(TimeSeries.merge(up_roll,down_roll), [:up,:down])
    rsi_ts = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_rsi(values)), updown)[:up],Symbol("RSI-",loopback))
    return rsi_ts
end

toRSI (generic function with 1 method)

In [7]:
function toORSI(price_ts,day0,day1)
    #pct = percentchange(price_ts)
    result_rsi = price_ts
    result_rsima = price_ts
    result_orsi = price_ts
    for i in day0:day1
        rsi_ts = toRSI(price_ts,i)
        result_rsi = merge(result_rsi,rsi_ts,method=:inner)
        for j in day0:day1
            rsi_ma = TimeSeries.rename(moving(mean,rsi_ts,j),Symbol("RSIMA-",i,"-",j))
            orsi = TimeSeries.rename(rsi_ts .- rsi_ma , Symbol("ORSI-",i,"-",j) )
            result_orsi = merge(result_orsi,orsi,method=:inner)
        end
    end
    #println(colnames(result_orsi))
    return result_orsi
end

toORSI (generic function with 1 method)

In [8]:
function toMatrixORSI(data,featureIdx,targetIdx)
    println(colnames(data)[1:5])
    #featureIdx=3
    data = values(data)
    X = data[:,featureIdx:end]
    #b = maximum(X)
    #a = minimum(X)
    #println(a,"-",b)
    #X = ( b .- X ) ./ (b-a+1)
    print(size(X))
    N = size(X)[1]
    k = Int(sqrt(size(X)[2]))
    println(k)
    println(X[1,1:4])
    println(X[2,1:4])
    println(X[2,k+1:k+4])
    X = transpose(X)
    X = reshape(X,(k,k,N))
    println(X[1:4,1,1])
    println(X[1:4,1,2])
    println(X[1:4,2,2])
    #targetIdx=1
    Y = data[:,targetIdx]
    return (X,Y)
end

toMatrixORSI (generic function with 1 method)

In [9]:
function toReturn(price_ts)
    pct = TimeSeries.rename(percentchange(price_ts),[:return])
    pct = moving(mean,pct,5)
    return TimeSeries.map((timestamp, values) -> (timestamp, 100*values), pct)
end   

toReturn (generic function with 1 method)

In [10]:
function preprocessing(etf,day0,day1,mlclass)
    PL = select(etf,[:Open,:Close,:ondate])
    price = select(etf,[:Close,:ondate])
    PL_ts = TimeArray(PL,timestamp=:ondate)
    PL_ts = toPL(PL_ts)
    pct = TimeSeries.rename(percentchange(TimeArray(price,timestamp=:ondate)),[:return])
    price_ts = TimeArray(price,timestamp=:ondate)
    pct = toReturn(price_ts)
    #day0=5
    #day1=30
    result_orsi = toORSI(price_ts,day0,day1)
    tgt = merge(PL_ts,pct,method=:inner)
    data = merge(tgt,result_orsi,method=:inner)
    featureIdx = 4
    if mlclass == :reg
        tgt = 2
    else
        tgt = 1
    end
        
    (X,Y) = toMatrixORSI(data,featureIdx,tgt)
end

preprocessing (generic function with 1 method)

In [11]:
function timeseriesDataset(X,Y,seqlen,mlclass)
    xtrain = Array{Float32,3}[]
    ytrain = Array{Float32,1}[]
    len = length(Y)
    for i in 1:len-seqlen-1
        mx = X[:,:,i:i+seqlen-1]
        xtrain = vcat(xtrain,[mx])
        if mlclass == :reg
            my = Y[i+seqlen]
        else
            my = Flux.onehot(Y[i+seqlen], [0,1])
        end
        ytrain = vcat(ytrain,[my])
    end
    xcurrent = [X[:,:,end-seqlen+1:end]]
    return (xtrain,ytrain,xcurrent)
end
    

timeseriesDataset (generic function with 1 method)

In [12]:
function build_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    #Dropout(0.1),
    Dense(sz*sz*Nh, 2),
    softmax
    )
end

build_model (generic function with 1 method)

In [13]:
function build_reg_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    Dropout(0.1),
    (x->transpose(x)),
    GRU(1,Nh),
    GRU(Nh,Nh),
    (x -> x[:,end]),
    Dense(Nh, 1),
    (x -> x[1])
    )
end

build_reg_model (generic function with 1 method)

In [14]:
function accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

accuracy (generic function with 1 method)

In [15]:
function reg_accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = prediction[i] > 0 ? 1 : 0
        y = ytest[i] > 0 ? 1 : 0
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

reg_accuracy (generic function with 1 method)

In [16]:
function plot_eval(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    #println(len)
    pred = Float32[]
    actual = Float32[]
    total = 0
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        append!(pred,yh)
        append!(actual,y)
    end
    plot(1:len,pred,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
    plot!(1:len,actual,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
end

plot_eval (generic function with 1 method)

In [17]:
function myTrain(etf,seqlen,Nh,lr,mm,day0,day1, mlclass,i)
    #seqlen=7
    #mlclass = :clf
    (X,Y) = preprocessing(etf,day0,day1,mlclass)
    (xtrain,ytrain,xcurrent) = timeseriesDataset(X,Y,seqlen,mlclass)
    thd = 150
    xtest,ytest = xtrain[end-thd+1:end], ytrain[end-thd+1:end]
    xtrain,ytrain = xtrain[1:end-thd], ytrain[1:end-thd]
    batchsize = 20
    train_loader = Flux.Data.DataLoader(xtrain,ytrain, batchsize=batchsize,shuffle=false)
    #Nh=10
    delta = day1 - day0 + 1
    if mlclass == :reg
        m = build_reg_model(Nh,seqlen,delta)
    else
        m = build_model(Nh,seqlen,delta)
    end
    function loss(x, y)    
        yh = m.(x)
        #println(yh)
        #println(y)
        #println(Flux.Losses.logitcrossentropy(yh[1], y[1]))
        return sum(Flux.Losses.logitcrossentropy.(yh, y))
    end
    function acc_loss(x,y)
        (acc,y0,y1,y2) = accuracy(m,x,y)
        return (1-acc-y0+y1)*abs(y2-0.5)
    end
    function mse_loss(x,y)
        yh = m.(x)
        e = Flux.mae(yh,y)
        return e
    end
    evalcb() = @show loss(xtest,ytest)
    num_epoch = 25
    #lr = 0.01
    if mlclass == :reg
        @epochs num_epoch Flux.train!(mse_loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = reg_accuracy(m,xtest,ytest)
        prediction = m.(xtest)
        println(size(prediction))
        mseloss = mse_loss(xtest,ytest)
        println("mse=",mseloss)
        objcost = mseloss-1*(confmx[1,1]+confmx[2,2]-confmx[1,2]-confmx[2,1])
        #p = map(x -> x[end],prediction)
        #y = map(x -> x[end],ytest)
        #lp = getindex(size(prediction),1)
        #ly = getindex(size(y),1)
        plot(1:length(prediction),prediction,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        plot!(1:length(ytest),ytest,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        dir="/Users/kahingleung/Downloads/gold-julia/"
        png("/Users/kahingleung/Downloads/gold-julia/plot-"*string(i)*".png")
        println("current size:", size(xcurrent[1]))
        future = m.(xcurrent)
        future = future[1]
        println("future:",future)
    else
        @epochs num_epoch Flux.train!(loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = accuracy(m,xtest,ytest)
        objcost = -1*(confmx[1,1]+confmx[2,2]-confmx[1,2]-confmx[2,1])
        println("current size:", size(xcurrent[1]))
        future = m.(xcurrent)
        future = future[1]
        println("future:",future)
    end
    println("accuracy:",(confmx[1,1]+confmx[2,2]))
    println(confmx[1,:])
    println(confmx[2,:])
    
    return (m,objcost,xtest,ytest,confmx,future)
end
function myObjective(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    (m,objcost,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    Flux.reset!(m)
    return objcost
end

myObjective (generic function with 1 method)

In [18]:
#sym="2840.HK"
sym="GLD"
etf = wgetStock(sym,3)

,Open,Close,High,Low,Volume,ondate
,Float32,Float32,Float32,Float32,Float32,DateTime
1,116.26,115.83,116.57,115.625,5.55043e6,2017-02-02T05:00:00
2,115.73,116.14,116.365,115.71,7.0282e6,2017-02-03T05:00:00
3,117.07,117.68,117.74,116.74,7.43806e6,2017-02-06T05:00:00
4,117.31,117.46,117.74,117.2,7.88362e6,2017-02-07T05:00:00
5,118.09,118.17,118.59,117.81,8.00184e6,2017-02-08T05:00:00
6,118.32,117.31,118.58,117.2,7.4162e6,2017-02-09T05:00:00
7,116.68,117.59,117.86,116.67,7.19955e6,2017-02-10T05:00:00
8,116.73,116.79,116.95,116.15,5.89761e6,2017-02-13T05:00:00
9,117.52,116.93,117.56,116.38,5.92856e6,2017-02-14T05:00:00


In [19]:
ho = @phyperopt for i=20,
            sampler = RandomSampler(), # This is default if none provided
            seqlen = StepRange(3, 5, 20),
            Nh = StepRange(5,3, 20),
            delta = StepRange(10,5, 25),
            #lr =  LinRange(1e-5,9e-5,10),
            lr =  exp10.(LinRange(-4,-3,10)),
            mm =  LinRange(0.75,0.95,5),
            day0 = StepRange(5,3, 10)
    println(i,"-",seqlen,"-",Nh,"-",lr,"-",mm,"-",day0,"-",delta)
   @show myObjective(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,i)
end

1-8-8-0.0001-0.75-5-15
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[-26.298267, -29.8308, -33.308826, -35.917355]
Float32[-15.130589, -20.129208, -23.732212, -27.269909]
Float32[-15.789494, -20.250973, -23.553295, -26.705486]
Float32[-26.298267, -29.8308, -33.308826, -35.917355]
Float32[-15.130589, -20.129208, -23.732212, -27.269909]
Float32[-15.789494, -20.250973, -23.553295, -26.705486]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.37065518
current size:(16, 16, 8)
future:0.12689376
accuracy:0.66
[0.16, 0.1]
[0.24, 0.5]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = 0.050655179023742614
2-8-20-0.00035938136638046257-0.95-8-20
[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(945, 441)21
Float32[-23.681374, -24.84034, -22.30069, -20.613304]
Float32[-36.68711, -38.814316, -40.34256, -38.44129]
Float32[-29.756733, -31.679882, -31.334164, -29.767635]
Float32[-23.681374, -24.84034, -22.30069, -20.613304]
Float32[-36.68711, -38.814316, -40.34256, -38.44129]
Float32[-29.756733, -31.679882, -31.334164, -29.767635]
21-3


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.2649669
current size:(21, 21, 8)
future:0.25900525
accuracy:0.8133333333333334
[0.2733333333333333, 0.06]
[0.12666666666666668, 0.54]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.3616997615496318
3-18-11-0.0001668100537200059-0.8999999999999999-5-15
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[-26.298267, -29.8308, -33.308826, -35.917355]
Float32[-15.130589, -20.129208, -23.732212, -27.269909]
Float32[-15.789494, -20.250973, -23.553295, -26.705486]
Float32[-26.298267, -29.8308, -33.308826, -35.917355]
Float32[-15.130589, -20.129208, -23.732212, -27.269909]
Float32[-15.789494, -20.250973, -23.553295, -26.705486]
16-2


...

┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.3247595
current size:(26, 26, 3)
future:0.062132858
accuracy:0.7333333333333334
[0.24, 0.10666666666666667]
[0.16, 0.49333333333333335]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.14190715352694194
20-18-20-0.0001291549665014884-0.8999999999999999-5-20
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(951, 441)21
Float32[20.730927, 21.86316, 24.684433, 26.19786]
Float32[6.3249054, 7.973793, 9.172096, 11.831802]
Float32[5.5981903, 6.84494, 11.854294, 13.680405]
Float32[20.730927, 21.86316, 24.684433, 26.19786]
Float32[6.3249054, 7.973793, 9.172096, 11.831802]
Float32[5.5981903, 6.84494, 11.854294, 13.680405]
21-3


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.31768763
current size:(21, 21, 18)
future:0.29003993
accuracy:0.76
[0.24, 0.08]
[0.16, 0.52]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.20231236934661867


Hyperoptimizer(20, (:seqlen, :Nh, :delta, :lr, :mm, :day0), (3:5:18, 5:3:20, 10:5:25, [0.0001, 0.0001291549665014884, 0.0001668100537200059, 0.00021544346900318823, 0.0002782559402207126, 0.00035938136638046257, 0.00046415888336127773, 0.0005994842503189409, 0.000774263682681127, 0.001], range(0.75, stop=0.95, length=5), 5:3:8), Any[(8, 8, 15, 0.0001, 0.75, 5), (8, 20, 20, 0.00035938136638046257, 0.95, 8), (18, 11, 15, 0.0001668100537200059, 0.8999999999999999, 5), (18, 17, 15, 0.000774263682681127, 0.8999999999999999, 8), (18, 17, 20, 0.00035938136638046257, 0.85, 5), (13, 11, 20, 0.00021544346900318823, 0.95, 5), (8, 8, 20, 0.001, 0.75, 8), (18, 11, 15, 0.00046415888336127773, 0.8999999999999999, 5), (3, 20, 20, 0.0005994842503189409, 0.8, 8), (8, 17, 15, 0.0001291549665014884, 0.85, 8), (8, 14, 25, 0.001, 0.85, 8), (18, 8, 25, 0.0002782559402207126, 0.75, 8), (3, 20, 20, 0.0001291549665014884, 0.8999999999999999, 5), (13, 17, 10, 0.0005994842503189409, 0.75, 5), (3, 17, 20, 0.000774

In [20]:
best_params, min_f = ho.minimizer, ho.minimum

((18, 8, 15, 0.000774263682681127, 0.85, 8), -0.39542171080907185)

In [21]:
seqlen=best_params[1]
Nh=best_params[2]
delta=best_params[3]
lr=best_params[4]
mm=best_params[5]
day0=best_params[6]
(m,acc,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,21)

[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(955, 256)16
Float32[-3.1852837, -5.642517, -8.969746, -11.801373]
Float32[-3.771904, -6.649624, -8.944256, -12.059349]
Float32[-2.7380104, -5.398884, -7.6367607, -10.638302]
Float32[-3.1852837, -5.642517, -8.969746, -11.801373]
Float32[-3.771904, -6.649624, -8.944256, -12.059349]
Float32[-2.7380104, -5.398884, -7.6367607, -10.638302]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.28181136
current size:(16, 16, 18)
future:0.35113513
accuracy:0.78
[0.24, 0.06]
[0.16, 0.54]


(Chain(#24, Conv((2, 2), 18=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 8=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 8=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dropout(0.1), #25, Recur(GRUCell(1, 8)), Recur(GRUCell(8, 8)), #26, Dense(8, 1), #27), -0.2781886434555053, Array{Float32,3}[[-0.09635925 2.8918 … -1.0637016 3.0336723; 0.73394775 5.2024612 … -0.8840332 2.6353912; … ; 10.631416 13.127041 … -5.165554 -1.3513412; 10.34824 12.429699 … -5.7208595 -1.9289284]

[-16.753708 -10.082802 … -7.626087 -4.8973846; -16.255142 -10.109737 … -7.2437325 -4.988304; … ; -7.1030235 -0.5813141 … -11.258198 -8.954861; -7.2305946 -1.0887299 … -11.718395 -9.56221]

[-20.181904 -12.591148 … -5.918293 -6.4605637; -20.349995 -12.98246 … -5.6262665 -6.4056892; … ; -12.81752 -4.8014755 … -9.323071 -10.324039; -12.667271 -4.881878 … -9.528545 -10.774139]

...

[12.034348 2.2265549 … -0.46053314 1.3062057; 13.

In [22]:
#20-13-20-0.000774263682681127-0.8-5-15
seqlen=13
Nh=20
delta=15
lr=0.000774
mm=0.8
day0=5
println(seqlen,"-",Nh,"-",lr,"-",mm,"-",day0,"-",delta)
df = DataFrame(symbol = String[], accuracy = Float32[], future = Float32[])
#list = ["FB","AAPL","AMZN","GOOG","NFLX","SQ","MTCH","AYX","ROKU","TTD"]
#sym="GLD"
filepath="list.txt"
list=readlines(filepath)
println(list)
for sym in list
    etf = wgetStock(sym,3)
    (m,cost,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,sym)
    acc = (confmx[1,1]+confmx[2,2])
    push!(df,[sym,acc,future])
end
df

13-20-0.000774-0.8-5-15
["FB", "AAPL", "AMZN", "GOOG", "GLD", "SQ", "MTCH", "AYX", "ROKU", "TTD"]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[10.94191, 17.212006, 20.885178, 25.44368]
Float32[-6.2500763, -1.9910355, 3.326439, 6.6097717]
Float32[2.9136581, 7.001236, 10.962151, 13.184029]
Float32[10.94191, 17.212006, 20.885178, 25.44368]
Float32[-6.2500763, -1.9910355, 3.326439, 6.6097717]
Float32[2.9136581, 7.001236, 10.962151, 13.184029]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.52581155
current size:(16, 16, 13)
future:0.96604663
accuracy:0.8200000000000001
[0.36, 0.05333333333333334]
[0.12666666666666668, 0.46]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[16.231041, 19.343964, 20.637276, 21.698761]
Float32[10.947784, 14.344818, 17.422882, 18.917511]
Float32[7.9365463, 11.998787, 14.303268, 14.393501]
Float32[16.231041, 19.343964, 20.637276, 21.698761]
Float32[10.947784, 14.344818, 17.422882, 18.917511]
Float32[7.9365463, 11.998787, 14.303268, 14.393501]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=1.1297481
current size:(16, 16, 13)
future:0.84658504
accuracy:0.7133333333333333
[0.18666666666666668, 0.06666666666666667]
[0.22, 0.5266666666666666]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[26.589058, 31.39251, 34.526936, 38.379845]
Float32[20.043045, 27.163559, 32.0569, 35.467377]
Float32[21.411003, 27.780403, 31.748081, 34.407066]
Float32[26.589058, 31.39251, 34.526936, 38.379845]
Float32[20.043045, 27.163559, 32.0569, 35.467377]
Float32[21.411003, 27.780403, 31.748081, 34.407066]
16-2


...

┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.90830284
current size:(16, 16, 13)
future:-0.07988189
accuracy:0.8466666666666667
[0.22, 0.09333333333333334]
[0.06, 0.6266666666666667]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[9.057148, 12.1297035, 14.0975685, 17.439037]
Float32[14.679604, 19.428112, 22.616821, 24.809887]
Float32[9.660088, 13.9197235, 17.061794, 18.130684]
Float32[9.057148, 12.1297035, 14.0975685, 17.439037]
Float32[14.679604, 19.428112, 22.616821, 24.809887]
Float32[9.660088, 13.9197235, 17.061794, 18.130684]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.8182405
current size:(16, 16, 13)
future:0.973173
accuracy:0.8733333333333333
[0.23333333333333334, 0.02666666666666667]
[0.1, 0.64]


,symbol,accuracy,future
,String,Float32,Float32
1,FB,0.82,0.966047
2,AAPL,0.713333,0.846585
3,AMZN,0.753333,0.837541
4,GOOG,0.833333,0.796344
5,GLD,0.82,0.126236
6,SQ,0.84,-0.966226
7,MTCH,0.76,-1.28291
8,AYX,0.726667,0.178811
9,ROKU,0.846667,-0.0798819


In [23]:
df

,symbol,accuracy,future
,String,Float32,Float32
1,FB,0.82,0.966047
2,AAPL,0.713333,0.846585
3,AMZN,0.753333,0.837541
4,GOOG,0.833333,0.796344
5,GLD,0.82,0.126236
6,SQ,0.84,-0.966226
7,MTCH,0.76,-1.28291
8,AYX,0.726667,0.178811
9,ROKU,0.846667,-0.0798819


In [24]:
using Pkg; Pkg.generate("OsRSIConv")